In [117]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}.')

Using cuda.


In [118]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn

In [119]:
# You don't need to understand this function for now.
def load_data_CIFAR10(batch_size, resize=None):
    """Download the CIFAR10 dataset and then load it into memory."""
    trans = [torchvision.transforms.ToTensor()]
    if resize:
        trans.insert(0, torchvision.transforms.Resize(resize))
    trans = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.CIFAR10(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.CIFAR10(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=2),
            torch.utils.data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=2))

In [120]:
batch_size = 64 # Defines the batch size
train_iter, test_iter = load_data_CIFAR10(batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [121]:
X, y = next(iter(train_iter)) # Requests the first training batch
print(X.size()) # 256 images per batch. Each image is represented by a 1 x 28 x 28 tensor (number of channels x height x width). The images are grayscale, so there is a single channel.
print(y.size())

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [122]:
class BlockGen(nn.Module):
    # Creating Intermediate Blocks
    # the length of param indicates the number of intermediate blocks
    # each parameter is then inside the param for convolutional layers
    def __init__(self, in_channels, out_channels, c, layers, paddings, strides, kernels):
        super(BlockGen, self).__init__()
        self.relu = nn.ReLU()
        master_model = []
        self.c = c
        self.layers = layers
        for i in range(c):
            convs = nn.ModuleList()
            for j in range(layers[i]):
                convs.append(nn.Conv2d(in_channels[i][j], out_channels[i][j], kernel_size=kernels[i][j],stride= strides[i][j], padding= paddings[i][j]))
            master_model.append(convs)
        
        self.model = master_model
        # the fully-connected layer turning m to a to calculate the weights of convolutional layer in the final equation the output should be the same value as the number of concolutional layers
       self.fc = nn.Linear(in_channels[0][0], c)
        
    def forward(self, x):
        m = []
        for i in range(x.size()[0]):
            m.append((float(x[i].flatten().mean())))
        m = torch.Tensor(m)
        a = self.fc(m)
        # creating the convolutional neurons in the layer of the block
        x = self.relu(x)
        model = self.model
        out_list = []
        for i in range(self.c):
            s.append(0)
            mod = mo[i]
            out = x
            for model in mod:
                out = model(out)
            print(out.size())
            out_list[i] = out
 
        return sum(out_list)
        

In [132]:
class FinalModel(nn.Module):
    # Creating Intermediate Blocks
    # the length of param indicates the number of intermediate blocks
    # each parameter is then inside the param for convolutional layers
    def __init__(self, num_block, c, layers, in_channels, out_channels, kernels, paddings, strides):
        super(FinalModel, self).__init__()
        blocks = nn.ModuleList
        for i in range(num_block):
            blocks.append(BlockGen(c=c[i], layers=layers[i], in_channels=in_channels[i], out_channels=out_channels[i], kernels=kernels[i], paddings=paddings[i], strides=strides[i]))
        self.blocks = blocks
        self.fc = nn.Linear(in_channels[-1][-1][-1], 10)
        
    def forward(self, x):
        # creating the convolutional neurons in the layer of the block
        x = self.relu(x)
        model = self.model
        out_list = []
        for model in self.blocks:
            x = model(x)
            
        for i in range(x.size()[0]):
            m.append((float(x[i].flatten().mean())))
        m = torch.Tensor(m)
        out = self.fc(m)
 
        return out

In [8]:
# Applies Xavier initialization if the `torch.nn.Module` is `torch.nn.Linear` or `torch.nn.Conv2d`
def init_weights(m):
    if type(m) == torch.nn.Linear or type(m) == torch.nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

num_outputs = 10
model = BlockGen(in_channels=[[3,12],[3,6]], out_channels=[[12,12],[6,12]], c=2, layers=[2,2], paddings=[[2,0],[0,0]], strides=[[1,2],[1,1]], kernels=[[3,2],[3,1]])
model.apply(init_weights) # Applies `init_weights` to every `torch.nn.Module` inside `model`

ConvBlock(
  (relu): ReLU()
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=10, out_features=1, bias=True)
)

In [9]:
loss = torch.nn.CrossEntropyLoss()

In [10]:
lr = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [11]:
def correct(logits, y):
    y_hat = logits.argmax(axis=1) # Finds the column with the highest value for each row of `logits`.
    return (y_hat == y).float().sum() # Computes the number of times that `y_hat` and `y` match.

In [12]:
def evaluate_metric(model, data_iter, metric):
    """Compute the average `metric` of the model on a dataset."""
    c = torch.tensor(0.)
    n = torch.tensor(0.)
    for X, y in data_iter:
        logits = model(X)
        c += metric(logits, y)
        n += len(y)

    return c / n

In [ ]:
print(f'Training accuracy: {evaluate_metric(model, train_iter, correct)}. Testing accuracy: {evaluate_metric(model, test_iter, correct)}.')

# Basic Code

In [ ]:
o = i

In [9]:
import torch
n = torch.Tensor([[[1,2,6,4,7],[2,4,3,5,7]],[[1,1,4,3,2],[2,2,7,5,7]], [[1,4,8,4,7],[2,4,8,5,7]]])
m = []
for i in range(n.size()[0]):
    m.append((float(n[i].flatten().mean())))
m = torch.Tensor(m)
m

tensor([4.1000, 3.4000, 5.0000])

In [3]:
class Model(torch.nn.Module):
    
    def __init__(self, ):
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(3, 5)
    def forward(self, x):
        out  = self.fc1(x)
        return out

In [4]:
k = Model()
out = k(m)

NameError: name 'm' is not defined

In [6]:
class OutBlock:

SyntaxError: incomplete input (2093479193.py, line 1)

In [109]:
param = [{'L': 3,'kernel_size':1 , 'stride':1, 'padding':0}, {'out':3, 'L': 3,'kernel_size':2 , 'stride':3, 'padding':1}, {'out':2, 'L': 5,'kernel_size':2 , 'stride':1, 'padding':0}]

In [110]:
for i, j in enumerate(param):
    print(param[i])

{'L': 3, 'kernel_size': 1, 'stride': 1, 'padding': 0}
{'out': 3, 'L': 3, 'kernel_size': 2, 'stride': 3, 'padding': 1}
{'out': 2, 'L': 5, 'kernel_size': 2, 'stride': 1, 'padding': 0}


In [8]:
mod

NameError: name 'mod' is not defined

In [7]:
mod = Blo"ckGen(param=param, in_channels=3, out_channels=10)

SyntaxError: unterminated string literal (detected at line 1) (4252390199.py, line 1)

In [113]:
out = mod(n)
len(out)

3

In [100]:
len(param)

3

## Test Example

In [12]:
n = torch.randn(3,6,6)
n

tensor([[[ 0.5916,  0.5676, -1.0415,  0.2531,  1.1415, -1.0663],
         [ 1.0914,  0.5812,  0.7329, -0.9620, -3.5427, -0.9107],
         [ 0.1481, -0.6351, -2.2436,  0.2195,  0.3712, -0.2928],
         [-0.3627, -0.1971, -1.1930,  0.7793, -0.3895,  1.8926],
         [ 0.6179,  1.7252, -1.0225, -0.3343, -0.7708, -1.1098],
         [-0.8818,  0.1688, -0.0045,  0.5856, -0.9097, -0.2356]],

        [[-0.9179,  0.8336, -0.8736, -0.5851,  0.9430,  0.1678],
         [ 1.1078,  0.6509,  0.2739,  0.2685,  0.1837, -0.4887],
         [ 0.1076, -1.4711, -0.1499,  1.0199,  1.2663,  1.3709],
         [-0.8724, -0.8335,  0.5277, -1.7724,  1.4742,  0.6399],
         [-0.3226,  0.7655,  0.9605, -0.1787,  0.4703, -1.9245],
         [-1.9839, -1.3509, -0.2614, -0.0918, -1.8255, -0.7549]],

        [[ 0.1341,  0.6445,  0.7236, -0.3718, -1.1775, -0.1703],
         [ 0.8173, -1.1339, -0.3453,  0.7949, -0.0057,  0.5891],
         [ 1.2043,  1.0888,  0.1014, -2.1153, -0.5242, -0.2361],
         [ 0.5098,  0

In [14]:
mo = nn.Conv2d(3, 12, kernel_size=3, padding= 2, stride=1)
out = mo(n)

In [16]:
out.size()

torch.Size([12, 8, 8])

In [17]:
mo2 = nn.Conv2d(12, 12, kernel_size=2, padding=0, stride=2)
out2 = mo2(out)
out2.size()

torch.Size([12, 4, 4])

In [85]:
n[2][1][0]*n

tensor([[[ 0.4835,  0.4639, -0.8512,  0.2068,  0.9329, -0.8715],
         [ 0.8920,  0.4750,  0.5990, -0.7862, -2.8954, -0.7443],
         [ 0.1210, -0.5190, -1.8336,  0.1794,  0.3034, -0.2393],
         [-0.2965, -0.1611, -0.9751,  0.6369, -0.3183,  1.5468],
         [ 0.5050,  1.4100, -0.8356, -0.2733, -0.6300, -0.9070],
         [-0.7207,  0.1379, -0.0037,  0.4786, -0.7435, -0.1926]],

        [[-0.7502,  0.6813, -0.7140, -0.4782,  0.7707,  0.1371],
         [ 0.9053,  0.5319,  0.2239,  0.2194,  0.1502, -0.3994],
         [ 0.0880, -1.2023, -0.1225,  0.8335,  1.0350,  1.1204],
         [-0.7130, -0.6812,  0.4312, -1.4486,  1.2049,  0.5230],
         [-0.2636,  0.6256,  0.7850, -0.1461,  0.3844, -1.5728],
         [-1.6214, -1.1041, -0.2136, -0.0750, -1.4920, -0.6170]],

        [[ 0.1096,  0.5267,  0.5914, -0.3038, -0.9623, -0.1392],
         [ 0.6680, -0.9267, -0.2822,  0.6497, -0.0046,  0.4815],
         [ 0.9842,  0.8898,  0.0829, -1.7288, -0.4284, -0.1930],
         [ 0.4166,  0

In [69]:
n

tensor([[[ 0.5916,  0.5676, -1.0415,  0.2531,  1.1415, -1.0663],
         [ 1.0914,  0.5812,  0.7329, -0.9620, -3.5427, -0.9107],
         [ 0.1481, -0.6351, -2.2436,  0.2195,  0.3712, -0.2928],
         [-0.3627, -0.1971, -1.1930,  0.7793, -0.3895,  1.8926],
         [ 0.6179,  1.7252, -1.0225, -0.3343, -0.7708, -1.1098],
         [-0.8818,  0.1688, -0.0045,  0.5856, -0.9097, -0.2356]],

        [[-0.9179,  0.8336, -0.8736, -0.5851,  0.9430,  0.1678],
         [ 1.1078,  0.6509,  0.2739,  0.2685,  0.1837, -0.4887],
         [ 0.1076, -1.4711, -0.1499,  1.0199,  1.2663,  1.3709],
         [-0.8724, -0.8335,  0.5277, -1.7724,  1.4742,  0.6399],
         [-0.3226,  0.7655,  0.9605, -0.1787,  0.4703, -1.9245],
         [-1.9839, -1.3509, -0.2614, -0.0918, -1.8255, -0.7549]],

        [[ 0.1341,  0.6445,  0.7236, -0.3718, -1.1775, -0.1703],
         [ 0.8173, -1.1339, -0.3453,  0.7949, -0.0057,  0.5891],
         [ 1.2043,  1.0888,  0.1014, -2.1153, -0.5242, -0.2361],
         [ 0.5098,  0

In [70]:
n[2][1][0]

tensor(0.8173)

In [71]:
fc = nn.Linear(3, 2)
m = []
for i in range(3):
    m.append((float(n[i].flatten().mean())))
m = torch.Tensor(m)
a = fc(m)
a

tensor([-0.1650,  0.0029], grad_fn=<ViewBackward0>)

In [72]:
a[0]*n[2]

tensor([[-0.0221, -0.1064, -0.1194,  0.0613,  0.1943,  0.0281],
        [-0.1349,  0.1871,  0.0570, -0.1312,  0.0009, -0.0972],
        [-0.1987, -0.1797, -0.0167,  0.3490,  0.0865,  0.0390],
        [-0.0841, -0.0912, -0.0561,  0.3345,  0.0583, -0.2240],
        [-0.3705, -0.1168,  0.0073, -0.2664,  0.2744, -0.0208],
        [-0.2223, -0.1158, -0.3274,  0.1549, -0.0873,  0.0766]],
       grad_fn=<MulBackward0>)

In [73]:
l = [n, n]
s = sum(l)
s.size()

torch.Size([3, 6, 6])

In [74]:
n

tensor([[[ 0.5916,  0.5676, -1.0415,  0.2531,  1.1415, -1.0663],
         [ 1.0914,  0.5812,  0.7329, -0.9620, -3.5427, -0.9107],
         [ 0.1481, -0.6351, -2.2436,  0.2195,  0.3712, -0.2928],
         [-0.3627, -0.1971, -1.1930,  0.7793, -0.3895,  1.8926],
         [ 0.6179,  1.7252, -1.0225, -0.3343, -0.7708, -1.1098],
         [-0.8818,  0.1688, -0.0045,  0.5856, -0.9097, -0.2356]],

        [[-0.9179,  0.8336, -0.8736, -0.5851,  0.9430,  0.1678],
         [ 1.1078,  0.6509,  0.2739,  0.2685,  0.1837, -0.4887],
         [ 0.1076, -1.4711, -0.1499,  1.0199,  1.2663,  1.3709],
         [-0.8724, -0.8335,  0.5277, -1.7724,  1.4742,  0.6399],
         [-0.3226,  0.7655,  0.9605, -0.1787,  0.4703, -1.9245],
         [-1.9839, -1.3509, -0.2614, -0.0918, -1.8255, -0.7549]],

        [[ 0.1341,  0.6445,  0.7236, -0.3718, -1.1775, -0.1703],
         [ 0.8173, -1.1339, -0.3453,  0.7949, -0.0057,  0.5891],
         [ 1.2043,  1.0888,  0.1014, -2.1153, -0.5242, -0.2361],
         [ 0.5098,  0

In [81]:
s = []
s.append(0)
s[0] = n
s

[tensor([[[ 0.5916,  0.5676, -1.0415,  0.2531,  1.1415, -1.0663],
          [ 1.0914,  0.5812,  0.7329, -0.9620, -3.5427, -0.9107],
          [ 0.1481, -0.6351, -2.2436,  0.2195,  0.3712, -0.2928],
          [-0.3627, -0.1971, -1.1930,  0.7793, -0.3895,  1.8926],
          [ 0.6179,  1.7252, -1.0225, -0.3343, -0.7708, -1.1098],
          [-0.8818,  0.1688, -0.0045,  0.5856, -0.9097, -0.2356]],
 
         [[-0.9179,  0.8336, -0.8736, -0.5851,  0.9430,  0.1678],
          [ 1.1078,  0.6509,  0.2739,  0.2685,  0.1837, -0.4887],
          [ 0.1076, -1.4711, -0.1499,  1.0199,  1.2663,  1.3709],
          [-0.8724, -0.8335,  0.5277, -1.7724,  1.4742,  0.6399],
          [-0.3226,  0.7655,  0.9605, -0.1787,  0.4703, -1.9245],
          [-1.9839, -1.3509, -0.2614, -0.0918, -1.8255, -0.7549]],
 
         [[ 0.1341,  0.6445,  0.7236, -0.3718, -1.1775, -0.1703],
          [ 0.8173, -1.1339, -0.3453,  0.7949, -0.0057,  0.5891],
          [ 1.2043,  1.0888,  0.1014, -2.1153, -0.5242, -0.2361],
    

In [116]:
class Test1(nn.Module):
    # Creating Intermediate Blocks
    # the length of param indicates the number of intermediate blocks
    # each parameter is then inside the param for convolutional layers
    def __init__(self, in_channels, out_channels, c, layers, paddings, strides, kernels):
        super(Test1, self).__init__()
        self.relu = nn.ReLU()
        master_model = []
        self.c = c
        self.layers = layers
        for i in range(c):
            convs = nn.ModuleList()
            for j in range(layers[i]):
                convs.append(nn.Conv2d(in_channels[i][j], out_channels[i][j], kernel_size=kernels[i][j],stride= strides[i][j], padding= paddings[i][j]))
            master_model.append(convs)
        
        self.model = master_model
        # the fully-connected layer turning m to a to calculate the weights of convolutional layer in the final equation the output should be the same value as the number of concolutional layers
        self.fc = nn.Linear(in_channels[0][0], c)
        
    def forward(self, x):
        m = []
        for i in range(x.size()[0]):
            m.append((float(x[i].flatten().mean())))
        m = torch.Tensor(m)
        a = self.fc(m)
        # creating the convolutional neurons in the layer of the block
        x = self.relu(x)
        model = self.model
        out_list = []
        for i in range(self.c):
            s.append(0)
            mod = mo[i]
            out = x
            for model in mod:
                out = model(out)
            print(out.size())
            out_list[i] = out
 
        return sum(out_list)

In [112]:
t1 = Test1(in_channels=[[3,12],[3,6]], out_channels=[[12,12],[6,12]], c=2, layers=[2,2], paddings=[[2,0],[0,0]], strides=[[1,2],[1,1]], kernels=[[3,2],[3,1]])
out= t1(n)


torch.Size([12, 4, 4])
torch.Size([12, 4, 4])


In [131]:
c[0]

2

In [133]:
num_blocks=2
c = [2,1]
in_channels = [[[3,12],[3,6]],[[12]]]
out_channels=[[[12,12],[6,12]], [[24]]]
layers = [[2,2], [2]]
paddings = [[[2,0],[0,0]],[[0]]]
strides= [[[1,2],[1,1]], [[1]]]
kernels = [[[3,2],[3,1]], [[2]]]
t2 = FinalModel(num_block=2, c = c, layers = layers, paddings = paddings, strides = strides, kernels= kernels, in_channels=in_channels, out_channels=out_channels)
out2 = t2(n)

TypeError: ModuleList.append() missing 1 required positional argument: 'module'

In [113]:
out.size()

torch.Size([12, 4, 4])

In [114]:
out

tensor([[[-7.8419e-02,  1.6770e-01,  3.1307e-01,  7.2142e-01],
         [ 3.3566e-01,  3.9943e-01,  3.3564e-01,  5.6603e-01],
         [ 3.6193e-01,  3.2218e-01,  3.7127e-01, -6.9696e-02],
         [ 5.0027e-01,  2.2803e-01, -3.7562e-01,  1.4719e-01]],

        [[-1.8553e-01, -4.0123e-01, -2.9772e-01,  2.7814e-01],
         [-8.0064e-02, -2.6071e-01, -1.5985e-01,  1.0840e-01],
         [-2.4959e-01, -2.0578e-01, -3.0418e-01,  6.0164e-02],
         [-3.8600e-01, -2.1963e-01, -2.9663e-01, -1.3248e-01]],

        [[ 5.4785e-01,  6.0923e-01,  3.0646e-01, -3.8727e-02],
         [ 8.1283e-02,  7.5408e-01,  8.9265e-01,  1.0795e+00],
         [ 5.9987e-01,  2.6477e-01,  2.0524e-01,  1.2205e+00],
         [ 3.9801e-01, -2.1707e-01,  3.4907e-01,  1.5826e-01]],

        [[ 6.0254e-01,  5.0626e-01,  3.3722e-01,  2.4083e-01],
         [ 4.6832e-01,  4.3215e-01,  8.5814e-02,  2.2329e-02],
         [ 4.4482e-01,  1.0624e-01,  1.5738e-01,  2.4904e-01],
         [ 4.6385e-01,  1.5108e-01,  2.7895e-01, 

In [6]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        
        self.fc = nn.Linear(out_channels, 1)
        
    def forward(self, x):
        out = self.relu(x)
        out1 = self.conv1(out)
        out2 = self.conv2(out)
        out3 = self.conv3(out)
        out4 = self.conv4(out)
        out5 = self.conv5(out)
        
        # Average pooling
        avg_pool = x.
        avg_pool = avg_pool.view(avg_pool.size(0), -1)
        # Weighted average calculation
        weights = torch.sigmoid(self.fc(avg_pool))
        print(weights.type())
        # Weighted sum
        weighted_sum = (out1 + out2 + out3 + out4 + out5) * weights.view(-1,1,1,1)
        
        return weighted_sum

In [123]:
a = [[[12,12],[6,12]],
    [[18, 24], [12, 24]]]

In [126]:
a[-1][-2][-1]

24